<a href="https://colab.research.google.com/github/dgsmith1988/ECSE-552-Final-Project/blob/main/execution_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overall Script Description
Run through each of the different cells to get things working. What should happen is that you connect to Google Drive, some data is transferred and then unzipped and then the model is trained on it and outputs data indicating correct operation. On the Google Drive there should also be files added under "Measurements" which have a date and time-stamp corresponding to the output of training process. The time stamps are generated by the Google Colab instance so they won't match exactly with your local time depending on where the instance they supplied us is located.

The cells are broken up based on functionality to make it easier to debug/profile/troubleshoot any issues which might come up.

In [ ]:
# Autoreload : automatically reload imported scripts so that changes on them take effect.
%load_ext autoreload
%autoreload 2

# Initial configurations

In [1]:
# Install required libraries. 
# NOTE: if an error is thrown, then please restart Runtime and run again!
!pip install torchmetrics
!pip install pytorch-lightning
!pip install librosa
!pip install lightning-bolts
!pip install optuna

# Configure Directories/Paths/Languages
The following cell is where you configure which languages you want to work with (from the three which are available on Google Drive at the moment). It is also where you set the paths to where the Google Drive will be mounted and where the data is extracted to.


In [1]:
import os
languages = ["EN", "DE", "ES"]
mount_point = '/content/drive'
load_path = os.path.join(mount_point, 'MyDrive/ECSE-552-FP/Data/')
unzip_path = '/content/speech_data'
save_path = '/MyDrive/ECSE-552-FP/Measurements'

from google.colab import drive
drive.mount(mount_point,force_remount= True)

Mounted at /content/drive


# Download and extract the dataset zip files
The following downloads and extracts the language zip files from Google Drive. It is done separately in it's own cell to make it easier to profile/debug things if there are issues in the network connection between Google Drive and Google Colab. 

This is separate from dataset creation for two reasons. The first is that if the network accesses are obscured by accessing the data via the network drive, it is much harder to debug bottlenecks in creating the dataset. The second is that all of the data must be transferred once at somepoint during one epoch, so it's easier to do it all up front and make sure the data is on the Colab instance before training. Localizing as much as possible to the Colab instance during training makes things easier to debug as well as more efficient.

There is a commented out code which allows you to select between the "debug" dataset and the "full" dataset for the languages. The "debug" set consists of a smaller subset (20 samples for each language) to aid in debugging the dataflow in models. It alleviates the need to run through the entire dataset and helps ensure all the pipes are connected correctly.

In [ ]:
for language in languages:
    language_dir_path = os.path.join(unzip_path, language)
    os.makedirs(language_dir_path, exist_ok=True)
    # archive = language + "_debug_set" + ".zip"
    archive = language + ".zip"
    !unzip {os.path.join(load_path, archive)} -d {language_dir_path}

Streaming output truncated to the last 5000 lines.
  inflating: /content/speech_data/EN/EagleX-20071225-npo/wav/b0211.wav  
   creating: /content/speech_data/EN/Edman274-20090103-fja/
   creating: /content/speech_data/EN/Edman274-20090103-fja/wav/
  inflating: /content/speech_data/EN/Edman274-20090103-fja/wav/b0062.wav  
  inflating: /content/speech_data/EN/Edman274-20090103-fja/wav/b0063.wav  
  inflating: /content/speech_data/EN/Edman274-20090103-fja/wav/b0064.wav  
  inflating: /content/speech_data/EN/Edman274-20090103-fja/wav/b0065.wav  
  inflating: /content/speech_data/EN/Edman274-20090103-fja/wav/b0066.wav  
  inflating: /content/speech_data/EN/Edman274-20090103-fja/wav/b0067.wav  
  inflating: /content/speech_data/EN/Edman274-20090103-fja/wav/b0068.wav  
  inflating: /content/speech_data/EN/Edman274-20090103-fja/wav/b0069.wav  
  inflating: /content/speech_data/EN/Edman274-20090103-fja/wav/b0070.wav  
  inflating: /content/speech_data/EN/Edman274-20090103-fja/wav/b0071.wav  
  

# Download the source code from GitHub

In [2]:
user = "dgsmith1988"
repo = "ECSE-552-Final-Project"
branch = "main"
src_dir = "Code"
pyfiles = ["models.py", "train.py", "dict_logger.py", "feature_extraction.py"]

for pyfile in pyfiles:
    !rm {pyfile}
    url = f"https://raw.githubusercontent.com/{user}/{repo}/{branch}/{src_dir}/{pyfile}"
    !wget {url}


--2022-04-02 01:09:40--  https://raw.githubusercontent.com/dgsmith1988/ECSE-552-Final-Project/main/Code/models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5779 (5.6K) [text/plain]
Saving to: ‘models.py’

models.py           100%[===================>]   5.64K  --.-KB/s    in 0s      

2022-04-02 01:09:40 (60.2 MB/s) - ‘models.py’ saved [5779/5779]

--2022-04-02 01:09:41--  https://raw.githubusercontent.com/dgsmith1988/ECSE-552-Final-Project/main/Code/train.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10769 (11K) [text/

# Hyperparameter tuning

In this section, we explore which set of hyperparameters gives the highest accuracy for the baseline model. The set of hyperparameters to analyse are :

- max_t : time to which all clips are to be cropped (seconds)
- bartch_size
- type of optimizer (Adam or SGD)

In [5]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-f668a3d9-147e-c62e-df8a-ebe4e4a43c72)


In [11]:
import optuna
import train

train.hp_tuning_voxforge_classifier(data_dir=unzip_path)

[I 2022-04-01 17:04:10,361] A new study created in memory with name: no-name-9f5b3212-c62e-4eff-b413-4d13245c76aa
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[W 2022-04-01 17:04:10,771] Trial 0 failed because of the following error: TypeError("__init__() missing 2 required positional arguments: 'rgb_expand' and 'crop'")
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/content/train.py", line 212, in objective
    dataset = AudioDataset(data_dir, max_t=max_t)
TypeError: __init__() missing 2 required positional arguments: 'rgb_expand' and 'crop'


Preparing and splitting dataset...


TypeError: ignored

# Run/Train the Model
This is broken into two parts:


1.   **Dataset/Model generation:** select your model and dataset parameters here
2.   **Training the model:** select your training parameters and train the model here

This was done as there can be substantial overhead in generating the dataset and often only needs to be done once. The separation makes it easier to test different training loop parameters iteratively. 

The functions here are just suggestions and if there is a better way to create the datasets or you find it easier to implement these functions in Colab while developing then by all means go ahead and do that. There are various sub-functions which you can call (i.e. *split_dataset()* or *init_dataset()*) which handle fairly common things and can be used outside the *get_datatsets()* function. For instance if you want to change the amount of data being used without modifying the file system organization (as copying data will cost time) you could *init_dataset()* to load the files in and then resize it and then call *split_dataset()* to get the appropriate train/validation split. Re-organization of the code can easily happen after things have been proven out.

Note: Model and dataset generation could easily be split apart if need be



In [5]:
from train import *
from models import *

model_name = MEL_AE_NAME
# model_name = BASELINE_RESNET_NAME

train_dataset = []
val_dataset = []
input_height = []

if model_name == BASELINE_RESNET_NAME:
    train_dataset, val_dataset = get_datasets(data_dir=unzip_path, dur_seconds=5, train_split=.8, crop=None, rgb_expand=False)
    model = BaselineResnetClassifier(num_classes=3)
elif model_name == MEL_AE_NAME:
    input_height = 128
    train_dataset, val_dataset = get_datasets(data_dir=unzip_path, dur_seconds=5, train_split=.8, crop=input_height, rgb_expand=True)
    model = Mel_ae(input_height, enc_type='resnet50', first_conv=False, maxpool1=False, enc_out_dim=2048, kl_coeff=0.1, latent_dim=3)

Creating dataset


3it [00:00,  8.37it/s]
3it [00:00,  8.98it/s]
3it [00:00,  8.87it/s]



Dataset creation in seconds:  1.0531604640000296


In [6]:
max_epoch = 5
# max_epoch = 20
batch_size = 10

train_model(model, model_name, train_dataset, val_dataset, max_epoch, batch_size)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /content/DictLogger/0.1/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type          | Params
------------------------------------------
0 | encoder | ResNetEncoder | 23.5 M
1 | decoder | ResNetDecoder | 8.3 M 
2 | fc      | Linear        | 6.1 K 
------------------------------------------
31.9 M    Trainable params
0         Non-trainable params
31.9 M    Total params
127.400   Total estimated model params size (MB)


Creating measurements path...
Running on Colab
Initializing trainer...


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1942: PossibleUserWarning: The number of training samples (5) is smaller than the logging interval Trainer(log_every_n_s

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# Save model to Google Drive

In [ ]:
# When a model needs to be saved for future analysis, it must be moved to Google drive, since colab is not-persistent

In [ ]:
# First, download the move_checkpoint.py script
user = "dgsmith1988"
repo = "ECSE-552-Final-Project"
src_dir = "Code"
pyfiles = ["move_checkpoint.py"]

for pyfile in pyfiles:
    !rm {pyfile}
    url = f"https://raw.githubusercontent.com/{user}/{repo}/rubert/{src_dir}/{pyfile}"
    !wget {url}

rm: cannot remove 'move_checkpoint.py': No such file or directory
--2022-03-19 16:44:25--  https://raw.githubusercontent.com/dgsmith1988/ECSE-552-Final-Project/rubert/Code/move_checkpoint.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1420 (1.4K) [text/plain]
Saving to: ‘move_checkpoint.py’

move_checkpoint.py  100%[===================>]   1.39K  --.-KB/s    in 0s      

2022-03-19 16:44:25 (23.8 MB/s) - ‘move_checkpoint.py’ saved [1420/1420]



In [ ]:
# NExt, move the desired file, and assigne a new meanigful_name. 
# it will be properly stored in the folder Checkpoints
from move_checkpoint import move_checkpoint
move_checkpoint(r'/content/DictLogger/0.1/checkpoints/epoch=4-step=10989.ckpt', 'first_step')

Running on Colab


In [1]:
# TODO: which hyperparameters parmeters should be stored?
# TODO: do the checkpoints contain the required hyperparameters of a particular model? If not, is there a way to save these hyperparameters?
# TODO: what are the best ways to find the optimal parameters? Cross validation?
# TODO: should I include other metrics beyond loss?

reassign: [0]
append: [0, 1]
